In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# question1
def load_data(filepath):
  df = pd.read_csv(filepath)
  df.head()
  Y = df['5'].to_numpy()
  del df['5']
  X = df.to_numpy()
  return X, Y

X, y = load_data("mnist_train.csv")
X_train, X_test, y_train, y_test = train_test_split(\
                X, y, test_size=0.3, random_state=42)
#One hot encoding of training labels 
Labels=pd.get_dummies(y_train)

In [10]:
# question 2
class Perceptron():
    def __init__(self,x,y):
        """
        x is 2d array of input images
        y are one hot encoded labels 
        """
        self.x=x/255   # Divide by 255 to normalise the pixel values (0-255)
        self.y=y
        self.weights=[]
        self.bias=[]
        self.outputs=[]
        self.derivatives=[]
        self.activations=[]
        
    def connect(self,layer1,layer2):
        """layer 2 of shape 1xn"""
        #Initialise weights,derivatives and activation lists
        self.derivatives.append(np.random.uniform(0,0.1,size=(layer1.shape[1]+1,layer2.shape[1])))
        self.weights.append(np.random.uniform(-1,1,size=(layer1.shape[1]+1,layer2.shape[1])))
        self.bias.append(np.random.uniform(-1,1,size=(layer1.shape[1]+1,layer2.shape[1])))


    def softmax(self,z):
        e=np.exp(z)
        return e/np.sum(e,axis=1).reshape(-1,1) 
    
    def max_log_likelihood(self,y_pred,y):
        """cross entropy"""
        return y*np.log(y_pred)
    
    def delta_mll(self,y,y_pred):
        """derivative of cross entropy"""
        #return y*(y_pred-1)
        return y_pred-y
    
    def forward_pass(self,x,y,weights,bias):
        cost=0
        self.outputs=[]
        for i in range(len(weights)):
            samples=len(x)
            ones_array=np.ones(samples).reshape(samples,1)
            self.outputs.append(x) #append without adding ones array
            z=np.dot(np.append(ones_array,x,axis=1),weights[i]+bias[i])
            x=self.softmax(z)
        self.outputs.append(x)
        self.y_pred=x
        temp=-self.max_log_likelihood(self.y_pred,y)
        cost=np.mean(np.sum(temp,axis=1))
        return cost
    
    
    def backward_pass(self,y,lr):
        for i in range(len(self.weights)-1,-1,-1):
            ones_array=np.ones(len(n.outputs[i])).reshape(len(n.outputs[i]),1)
            prev_term=self.delta_mll(y,self.y_pred)  
            # derivatives follow specific order,last three terms added new,rest from previous term  
            self.derivatives[i]=np.dot(prev_term.T,np.append(ones_array,self.outputs[i],axis=1))   
            self.weights[i]=self.weights[i]-lr*((self.derivatives[i].T)/len(y))
            self.bias[i]=self.bias[i]-lr*((self.derivatives[i].T)/len(y))
    
    def train(self,batches,lr=1e-3,epoch=10):
        """number of batches to split data in,Learning rate and epochs"""
        for epochs in range(epoch):
            samples=len(self.x)
            c=0
            for i in range(batches):
              x_batch=self.x[int((samples/batches)*i):int((samples/batches)*(i+1))]
              y_batch=self.y.loc[int((samples/batches)*i):int((samples/batches)*(i+1))-1]
              
              c=self.forward_pass(x_batch,y_batch,self.weights,self.bias)
              self.backward_pass(y_batch,lr)
            print(epochs,c/batches)
    
    def predict(self,x):
        """input: x_test values"""
        x=x/255
        for i in range(len(self.weights)):
            samples=len(x)
            ones_array=np.ones(samples).reshape(samples,1)
            z=np.dot(np.append(ones_array,x,axis=1),self.weights[i]+self.bias[i])
            x=self.softmax(z) 
        return np.argmax(x,axis=1)


In [13]:
n=Perceptron(X_train,Labels)
n.connect(X_train,Labels)
n.train(batches=1000,lr=0.2,epoch=30)

0 0.0015373832867067966
1 0.0014641601415438855
2 0.0013973295334736115
3 0.0011700113684011207
4 0.0011865572495212452
5 0.0012489353875638411
6 0.0011753742870572596
7 0.0010465236922255399
8 0.0010842991486821076
9 0.0009344390620956329
10 0.0007138611606145924
11 0.0005668255123983662
12 0.0003678119470801297
13 0.0002808337801169118
14 0.000250976357750367
15 0.0002026352864272732
16 0.00016389797685827814
17 0.00013668304620253644
18 0.00010793064852517565
19 9.84301600801705e-05
20 8.950101116080666e-05
21 7.75720135575167e-05
22 7.084946772819047e-05
23 6.525940232625721e-05
24 6.143230120543388e-05
25 5.746171615042399e-05
26 5.600674684217869e-05
27 5.670239326887923e-05
28 5.882151232259818e-05
29 6.11191994956797e-05


In [14]:
pred=n.predict(X_test)
np.bincount(n.predict(X_test)),np.bincount(y_test)

(array([315, 330, 308, 334, 327, 285, 278, 320, 215, 288], dtype=int64),
 array([296, 327, 305, 326, 305, 283, 282, 336, 252, 288], dtype=int64))

In [15]:
print(f"accuracy is {np.bincount(np.abs(y_test-pred))[0]*100/len(y_test)} %")


accuracy is 88.8 %


In [5]:
# question3
class Layer():
    """
    size: Number of nodes in the hidden layer 
    activation: name of activation function for the layer
    """
    def __init__(self,size,activation='sigmoid'): 
        self.shape=(1,size)
        self.activation=activation
                
class SingleLayerNeuralNetwork():
    def __init__(self,x,y):
        """
        x is 2d array of input images
        y are one hot encoded labels 
        """
        self.x=x/255   # Divide by 255 to normalise the pixel values (0-255)
        self.y=y
        self.weights=[]
        self.bias=[]
        self.outputs=[]
        self.derivatives=[]
        self.activations=[]
        
    def connect(self,layer1,layer2):
        """layer 2 of shape 1xn"""
        #Initialise weights,derivatives and activation lists
        self.derivatives.append(np.random.uniform(0,0.1,size=(layer1.shape[1]+1,layer2.shape[1])))
        self.weights.append(np.random.uniform(-1,1,size=(layer1.shape[1]+1,layer2.shape[1])))
        self.bias.append(np.random.uniform(-1,1,size=(layer1.shape[1]+1,layer2.shape[1])))
        if isinstance(layer2,Layer):
            self.activations.append(layer2.activation)
            
    def activation(self,name,z,derivative=False):
        
        #implementation of various activation functions and their derivatives
        if name=='sigmoid':
            if derivative==False:
                return 1/(1+np.exp(-z))
            else:
                return z*(1-z)
        
    def softmax(self,z):
        e=np.exp(z)
        return e/np.sum(e,axis=1).reshape(-1,1) 
    
    def max_log_likelihood(self,y_pred,y):
        """cross entropy"""
        return y*np.log(y_pred)
    
    def delta_mll(self,y,y_pred):
        """derivative of cross entropy"""
        #return y*(y_pred-1)
        return y_pred-y
    
    def forward_pass(self,x,y,weights,bias):
        cost=0
        self.outputs=[]
        for i in range(len(weights)):
            samples=len(x)
            ones_array=np.ones(samples).reshape(samples,1)
            self.outputs.append(x) #append without adding ones array
            z=np.dot(np.append(ones_array,x,axis=1),weights[i]+bias[i])
            if i==len(weights)-1:
                x=self.softmax(z)
            else:
                x=self.activation(self.activations[i],z)
        self.outputs.append(x)
        self.y_pred=x
        
        temp=-self.max_log_likelihood(self.y_pred,y)
        cost=np.mean(np.sum(temp,axis=1))
        return cost
    
    
    def backward_pass(self,y,lr):
        for i in range(len(self.weights)-1,-1,-1):
            ones_array=np.ones(len(n.outputs[i])).reshape(len(n.outputs[i]),1)
            if i==len(self.weights)-1:
                prev_term=self.delta_mll(y,self.y_pred)  
                # derivatives follow specific order,last three terms added new,rest from previous term  
                self.derivatives[i]=np.dot(prev_term.T,np.append(ones_array,self.outputs[i],axis=1))   
            else:
                prev_term=np.dot(prev_term,self.weights[i+1][1:].T)*self.activation(self.activations[i],self.outputs[i+1],derivative=True)
                self.derivatives[i]=np.dot(prev_term.T,np.append(ones_array,self.outputs[i],axis=1))
            self.weights[i]=self.weights[i]-lr*((self.derivatives[i].T)/len(y))
            self.bias[i]=self.bias[i]-lr*((self.derivatives[i].T)/len(y))
                
    
    def train(self,batches,lr=1e-3,epoch=10):
        """number of batches to split data in,Learning rate and epochs"""
        for epochs in range(epoch):
            samples=len(self.x)
            c=0
            for i in range(batches):
              x_batch=self.x[int((samples/batches)*i):int((samples/batches)*(i+1))]
              y_batch=self.y.loc[int((samples/batches)*i):int((samples/batches)*(i+1))-1]
              
              c=self.forward_pass(x_batch,y_batch,self.weights,self.bias)
              self.backward_pass(y_batch,lr)
            print(epochs,c/batches)
    
    def predict(self,x):
        """input: x_test values"""
        x=x/255
        for i in range(len(self.weights)):
            samples=len(x)
            ones_array=np.ones(samples).reshape(samples,1)
            z=np.dot(np.append(ones_array,x,axis=1),self.weights[i]+self.bias[i])
            if i==len(self.weights)-1:
                x=self.softmax(z)
            else:
                x=self.activation(self.activations[i],z)

        return np.argmax(x,axis=1)


In [ ]:

n=SingleLayerNeuralNetwork(X_train,Labels)
l1=Layer(100)
n.connect(X_train,l1)
n.connect(l1,Labels)
n.train(batches=1000,lr=0.1,epoch=50)

0 0.0007244222394535747
1 0.0003677269832465923
2 0.00016489818961556115
3 0.0001273821077311397
4 0.00013661902539157614
5 0.000145652677640878
6 0.0001310232131454275
7 9.886985731332751e-05
8 6.824943936169827e-05
9 5.177598870312134e-05
10 4.343820046921563e-05
11 3.853947512926637e-05
12 3.5040965804228994e-05


In [ ]:
pred=n.predict(X_test)
np.bincount(n.predict(X_test)),np.bincount(y_test)

In [ ]:
print(f"accuracy is {np.bincount(np.abs(y_test-pred))[0]*100/len(y_test)} %")

In [ ]:
# question 4
class Layer():
    """
    size: Number of nodes in the hidden layer 
    activation: name of activation function for the layer
    """
    def __init__(self,size,activation='sigmoid'): 
        self.shape=(1,size)
        self.activation=activation
                
class DoubleLayerNeuralNetwork():
    def __init__(self,x,y):
        """
        x is 2d array of input images
        y are one hot encoded labels 
        """
        self.x=x/255   # Divide by 255 to normalise the pixel values (0-255)
        self.y=y
        self.weights=[]
        self.bias=[]
        self.outputs=[]
        self.derivatives=[]
        self.activations=[]
        
    def connect(self,layer1,layer2):
        """layer 2 of shape 1xn"""
        #Initialise weights,derivatives and activation lists
        self.derivatives.append(np.random.uniform(0,0.1,size=(layer1.shape[1]+1,layer2.shape[1])))
        self.weights.append(np.random.uniform(-1,1,size=(layer1.shape[1]+1,layer2.shape[1])))
        self.bias.append(np.random.uniform(-1,1,size=(layer1.shape[1]+1,layer2.shape[1])))
        if isinstance(layer2,Layer):
            self.activations.append(layer2.activation)
            
    def activation(self,name,z,derivative=False):
        
        #implementation of various activation functions and their derivatives
        if name=='sigmoid':
            if derivative==False:
                return 1/(1+np.exp(-z))
            else:
                return z*(1-z)
        
    def softmax(self,z):
        e=np.exp(z)
        return e/np.sum(e,axis=1).reshape(-1,1) 
    
    def max_log_likelihood(self,y_pred,y):
        """cross entropy"""
        return y*np.log(y_pred)
    
    def delta_mll(self,y,y_pred):
        """derivative of cross entropy"""
        #return y*(y_pred-1)
        return y_pred-y
    
    def forward_pass(self,x,y,weights,bias):
        cost=0
        self.outputs=[]
        for i in range(len(weights)):
            samples=len(x)
            ones_array=np.ones(samples).reshape(samples,1)
            self.outputs.append(x) #append without adding ones array
            z=np.dot(np.append(ones_array,x,axis=1),weights[i]+bias[i])
            if i==len(weights)-1:
                x=self.softmax(z)
            else:
                x=self.activation(self.activations[i],z)
        self.outputs.append(x)
        self.y_pred=x
        
        temp=-self.max_log_likelihood(self.y_pred,y)
        cost=np.mean(np.sum(temp,axis=1))
        return cost
    
    
    def backward_pass(self,y,lr):
        for i in range(len(self.weights)-1,-1,-1):
            ones_array=np.ones(len(n.outputs[i])).reshape(len(n.outputs[i]),1)
            if i==len(self.weights)-1:
                prev_term=self.delta_mll(y,self.y_pred)  
                # derivatives follow specific order,last three terms added new,rest from previous term  
                self.derivatives[i]=np.dot(prev_term.T,np.append(ones_array,self.outputs[i],axis=1))   
            else:
                prev_term=np.dot(prev_term,self.weights[i+1][1:].T)*self.activation(self.activations[i],self.outputs[i+1],derivative=True)
                self.derivatives[i]=np.dot(prev_term.T,np.append(ones_array,self.outputs[i],axis=1))
            self.weights[i]=self.weights[i]-lr*((self.derivatives[i].T)/len(y))
            self.bias[i]=self.bias[i]-lr*((self.derivatives[i].T)/len(y))
                
    
    def train(self,batches,lr=1e-3,epoch=10):
        """number of batches to split data in,Learning rate and epochs"""
        for epochs in range(epoch):
            samples=len(self.x)
            c=0
            for i in range(batches):
              x_batch=self.x[int((samples/batches)*i):int((samples/batches)*(i+1))]
              y_batch=self.y.loc[int((samples/batches)*i):int((samples/batches)*(i+1))-1]
              
              c=self.forward_pass(x_batch,y_batch,self.weights,self.bias)
              self.backward_pass(y_batch,lr)
            print(epochs,c/batches)
    
    def predict(self,x):
        """input: x_test values"""
        x=x/255
        for i in range(len(self.weights)):
            samples=len(x)
            ones_array=np.ones(samples).reshape(samples,1)
            z=np.dot(np.append(ones_array,x,axis=1),self.weights[i]+self.bias[i])
            if i==len(self.weights)-1:
                x=self.softmax(z)
            else:
                x=self.activation(self.activations[i],z)
        return np.argmax(x,axis=1)


In [ ]:
n=DoubleLayerNeuralNetwork(X_train,Labels)
l1=Layer(100)
l2=Layer(100)
n.connect(X_train,l1)
n.connect(l1,l2)
n.connect(l2,Labels)
n.train(batches=1000,lr=0.1,epoch=20)

In [ ]:
pred=n.predict(X_test)
np.bincount(n.predict(X_test)),np.bincount(y_test)

In [ ]:
print(f"accuracy is {np.bincount(np.abs(y_test-pred))[0]*100/len(y_test)} %")


In [ ]:
# question5
class Layer():
    """
    size: Number of nodes in the hidden layer 
    activation: name of activation function for the layer
    """
    def __init__(self,size,activation='sigmoid'): 
        self.shape=(1,size)
        self.activation=activation
                
class NeuralNetworkActivations():
    def __init__(self,x,y):
        """
        x is 2d array of input images
        y are one hot encoded labels 
        """
        self.x=x/255   # Divide by 255 to normalise the pixel values (0-255)
        self.y=y
        self.weights=[]
        self.bias=[]
        self.outputs=[]
        self.derivatives=[]
        self.activations=[]
        
    def connect(self,layer1,layer2):
        """layer 2 of shape 1xn"""
        #Initialise weights,derivatives and activation lists
        self.derivatives.append(np.random.uniform(0,0.1,size=(layer1.shape[1]+1,layer2.shape[1])))
        self.weights.append(np.random.uniform(-1,1,size=(layer1.shape[1]+1,layer2.shape[1])))
        self.bias.append(np.random.uniform(-1,1,size=(layer1.shape[1]+1,layer2.shape[1])))
        if isinstance(layer2,Layer):
            self.activations.append(layer2.activation)
            
    def activation(self,name,z,derivative=False):
        
        #implementation of various activation functions and their derivatives
        if name=='sigmoid':
            if derivative==False:
                return 1/(1+np.exp(-z))
            else:
                return z*(1-z)
        elif name=='relu':
            if derivative==False:
                return np.maximum(0.0,z)
            else:
              z[z<=0] = 0.0
              z[z>0] = 1.0
              return z
        elif name=='tanh':
          if derivative==False:
                return np.tanh(z)
          else:
                return 1.0 - (np.tanh(z)) ** 2
        
    def softmax(self,z):
        e=np.exp(z)
        return e/np.sum(e,axis=1).reshape(-1,1) 
    
    def max_log_likelihood(self,y_pred,y):
        """cross entropy"""
        return y*np.log(y_pred)
    
    def delta_mll(self,y,y_pred):
        """derivative of cross entropy"""
        #return y*(y_pred-1)
        return y_pred-y
    
    def forward_pass(self,x,y,weights,bias):
        cost=0
        self.outputs=[]
        for i in range(len(weights)):
            samples=len(x)
            ones_array=np.ones(samples).reshape(samples,1)
            self.outputs.append(x) #append without adding ones array
            z=np.dot(np.append(ones_array,x,axis=1),weights[i]+bias[i])
            if i==len(weights)-1:
                x=self.softmax(z)
            else:
                x=self.activation(self.activations[i],z)
        self.outputs.append(x)
        self.y_pred=x
        
        temp=-self.max_log_likelihood(self.y_pred,y)
        cost=np.mean(np.sum(temp,axis=1))
        return cost
    
    
    def backward_pass(self,y,lr):
        for i in range(len(self.weights)-1,-1,-1):
            ones_array=np.ones(len(n.outputs[i])).reshape(len(n.outputs[i]),1)
            if i==len(self.weights)-1:
                prev_term=self.delta_mll(y,self.y_pred)  
                # derivatives follow specific order,last three terms added new,rest from previous term  
                self.derivatives[i]=np.dot(prev_term.T,np.append(ones_array,self.outputs[i],axis=1))   
            else:
                prev_term=np.dot(prev_term,self.weights[i+1][1:].T)*self.activation(self.activations[i],self.outputs[i+1],derivative=True)
                self.derivatives[i]=np.dot(prev_term.T,np.append(ones_array,self.outputs[i],axis=1))
            self.weights[i]=self.weights[i]-lr*((self.derivatives[i].T)/len(y))
            self.bias[i]=self.bias[i]-lr*((self.derivatives[i].T)/len(y))
    
    def train(self,batches,lr=1e-3,epoch=10):
        """number of batches to split data in,Learning rate and epochs"""
        for epochs in range(epoch):
            samples=len(self.x)
            c=0
            for i in range(batches):
              x_batch=self.x[int((samples/batches)*i):int((samples/batches)*(i+1))]
              y_batch=self.y.loc[int((samples/batches)*i):int((samples/batches)*(i+1))-1]
              
              c=self.forward_pass(x_batch,y_batch,self.weights,self.bias)
              self.backward_pass(y_batch,lr)
            print(epochs,c/batches)
    
    def predict(self,x):
        """input: x_test values"""
        x=x/255
        for i in range(len(self.weights)):
            samples=len(x)
            ones_array=np.ones(samples).reshape(samples,1)
            z=np.dot(np.append(ones_array,x,axis=1),self.weights[i]+self.bias[i])
            if i==len(self.weights)-1:
                x=self.softmax(z)
            else:
                x=self.activation(self.activations[i],z)
        return np.argmax(x,axis=1)


In [ ]:
n=NeuralNetworkActivations(X_train,Labels)
l1=Layer(100,'sigmoid')
l2=Layer(50, 'tanh')
n.connect(X_train,l1)
n.connect(l1,l2)
n.connect(l2,Labels)
n.train(batches=1000,lr=0.1,epoch=20)

In [ ]:
pred=n.predict(X_test)
np.bincount(n.predict(X_test)),np.bincount(y_test)

In [ ]:
print(f"accuracy is {np.bincount(np.abs(y_test-pred))[0]*100/len(y_test)} %")

In [ ]:
# question6
class Layer():
    """
    size: Number of nodes in the hidden layer 
    activation: name of activation function for the layer
    """
    def __init__(self,size,activation='sigmoid'): 
        self.shape=(1,size)
        self.activation=activation
                
class NeuralNetworkMomentum():
    def __init__(self,x,y):
        """
        x is 2d array of input images
        y are one hot encoded labels 
        """
        self.x=x/255   # Divide by 255 to normalise the pixel values (0-255)
        self.y=y
        self.weights=[]
        self.bias=[]
        self.outputs=[]
        self.derivatives=[]
        self.activations=[]
        self.delta_weights=[]
        self.delta_bias=[]
        
    def connect(self,layer1,layer2):
        """layer 2 of shape 1xn"""
        #Initialise weights,derivatives and activation lists
        self.derivatives.append(np.random.uniform(0,0.1,size=(layer1.shape[1]+1,layer2.shape[1])))
        self.weights.append(np.random.uniform(-1,1,size=(layer1.shape[1]+1,layer2.shape[1])))
        self.bias.append(np.random.uniform(-1,1,size=(layer1.shape[1]+1,layer2.shape[1])))
        self.delta_weights.append(np.zeros((layer1.shape[1]+1,layer2.shape[1])))
        self.delta_bias.append(np.zeros((layer1.shape[1]+1,layer2.shape[1])))
        if isinstance(layer2,Layer):
            self.activations.append(layer2.activation)
            
    def activation(self,name,z,derivative=False):
        
        #implementation of various activation functions and their derivatives
        if name=='sigmoid':
            if derivative==False:
                return 1/(1+np.exp(-z))
            else:
                return z*(1-z)
        elif name=='relu':
            if derivative==False:
                return np.maximum(0.0,z)
            else:
              z[z<=0] = 0.0
              z[z>0] = 1.0
              return z
        elif name=='tanh':
          if derivative==False:
                return np.tanh(z)
          else:
                return 1.0 - (np.tanh(z)) ** 2
        
    def softmax(self,z):
        e=np.exp(z)
        return e/np.sum(e,axis=1).reshape(-1,1) 
    
    def max_log_likelihood(self,y_pred,y):
        """cross entropy"""
        return y*np.log(y_pred)
    
    def delta_mll(self,y,y_pred):
        """derivative of cross entropy"""
        #return y*(y_pred-1)
        return y_pred-y
    
    def forward_pass(self,x,y,weights,bias):
        cost=0
        self.outputs=[]
        for i in range(len(weights)):
            samples=len(x)
            ones_array=np.ones(samples).reshape(samples,1)
            self.outputs.append(x) #append without adding ones array
            z=np.dot(np.append(ones_array,x,axis=1),weights[i]+bias[i])
            if i==len(weights)-1:
                x=self.softmax(z)
            else:
                x=self.activation(self.activations[i],z)
        self.outputs.append(x)
        self.y_pred=x
        
        temp=-self.max_log_likelihood(self.y_pred,y)
        cost=np.mean(np.sum(temp,axis=1))
        return cost
    
    
    def backward_pass(self,y,lr,momentum=False,beta=0.9):
        for i in range(len(self.weights)-1,-1,-1):
            ones_array=np.ones(len(n.outputs[i])).reshape(len(n.outputs[i]),1)
            if i==len(self.weights)-1:
                prev_term=self.delta_mll(y,self.y_pred)  
                # derivatives follow specific order,last three terms added new,rest from previous term  
                self.derivatives[i]=np.dot(prev_term.T,np.append(ones_array,self.outputs[i],axis=1))   
            else:
                prev_term=np.dot(prev_term,self.weights[i+1][1:].T)*self.activation(self.activations[i],self.outputs[i+1],derivative=True)
                self.derivatives[i]=np.dot(prev_term.T,np.append(ones_array,self.outputs[i],axis=1))
            if momentum:
                self.delta_weights[i]=beta*self.delta_weights[i]-lr*((self.derivatives[i].T)/len(y))
                self.delta_bias[i]=beta*self.delta_bias[i]-lr*((self.derivatives[i].T)/len(y))
                self.weights[i]=self.weights[i]+self.delta_weights[i]
                self.bias[i]=self.bias[i]+self.delta_bias[i]
            else:
                self.weights[i]=self.weights[i]-lr*((self.derivatives[i].T)/len(y))
                self.bias[i]=self.bias[i]-lr*((self.derivatives[i].T)/len(y))
    
    def train(self,batches,lr=1e-3,epoch=10,beta):
        """number of batches to split data in,Learning rate and epochs"""
        for epochs in range(epoch):
            samples=len(self.x)
            c=0
            for i in range(batches):
              x_batch=self.x[int((samples/batches)*i):int((samples/batches)*(i+1))]
              y_batch=self.y.loc[int((samples/batches)*i):int((samples/batches)*(i+1))-1]
              
              c=self.forward_pass(x_batch,y_batch,self.weights,self.bias)
              self.backward_pass(y_batch,lr,momentum=True,beta)
            print(epochs,c/batches)
    
    def predict(self,x):
        """input: x_test values"""
        x=x/255
        for i in range(len(self.weights)):
            samples=len(x)
            ones_array=np.ones(samples).reshape(samples,1)
            z=np.dot(np.append(ones_array,x,axis=1),self.weights[i]+self.bias[i])
            if i==len(self.weights)-1:
                x=self.softmax(z)
            else:
                x=self.activation(self.activations[i],z)
        return np.argmax(x,axis=1)